In [3]:
import numpy as np
import pandas as pd 
import plotly.express as px

def plot_pca(pca_df, colour_column, cohort_columns, dataset,  x='PC1',y='PC2',z='PC3', color_mapping=None, height=500, width=750):
    fig= px.scatter_3d(
        pca_df, 
        x=x, 
        y=y,
        z=z, 
        title=f"PCA {dataset} | PC1 vs PC2 vs PC3 coloured by {colour_column}",
        color=colour_column, 
        hover_data=cohort_columns + ['sample_id'],
        color_discrete_map=color_mapping[colour_column], 
        template='simple_white',
        height=height,
        width=width
    )

    return fig
    

In [9]:
dataset = 'ag-vampir-002'
vcf_path = f"../../results/vcfs/amplicons/{dataset}.annot.vcf"
metadata_path = "../..//results/config/metadata.qcpass.tsv"
cohort_cols = 'location,pca_taxon'
wkdir = "../../"

In [10]:
import sys
import os
sys.path.append(os.path.join(wkdir, 'workflow'))
import ampseekertools as amp

## Population structure

In this notebook, we run a principal components analysis and build a neighbour joining tree on the amplicon sequencing variant data. For the PCA, we will plot PC1 v PC2 and PC3 v PC4, and the variance explained by the model.

In [11]:
cohort_cols = cohort_cols.split(",")

if metadata_path.endswith('.xlsx'):
    metadata = pd.read_excel(metadata_path, engine='openpyxl')
elif metadata_path.endswith('.tsv'):
    metadata = pd.read_csv(metadata_path, sep="\t")
elif metadata_path.endswith('.csv'):
    metadata = pd.read_csv(metadata_path, sep=",")
else:
    raise ValueError("Metadata file must be .xlsx or .csv")

import json
with open(f"{wkdir}/results/config/metadata_colours.json", 'r') as f:
    color_mapping = json.load(f)

geno, pos, contig, metadata, ref, alt, ann = amp.load_vcf(vcf_path, metadata)
df_pca, model = amp.pca(geno=geno, metadata=metadata, n_components=4)

/home/snagi/lstm_projects/ampseq-agvampir002/.snakemake/conda/3b1c9fccc6ddcee649a430b64373b071_/lib/python3.12/site-packages/allel/io/vcf_read.py:1732: UserWarning:

invalid INFO header: '##INFO=<ID=VDB,Number=1,Type=Float,Description="Variant Distance Bias for filtering splice-site artefacts in RNA-seq data (bigger is better)",Version="3">\n'



removing any invariant and highly missing sites


#### Variance explained

As a general rule of thumb, when the variance explained for each PC begins to flatten out, that is when the PCs are no longer informative.

In [12]:
fig = px.bar(model.explained_variance_ratio_ , labels={
                     "value": "Variance Explained",
                     "index": "Principal Component",
                 }, template='simple_white', height=250, width=600)
fig.update_layout(showlegend=False)

fig.show()

### PCA

In [18]:
for coh in cohort_cols:
    fig1 = plot_pca(df_pca, x='PC1',y='PC2',z='PC3', colour_column=coh, cohort_columns=cohort_cols, dataset=dataset, color_mapping=color_mapping)
    fig1.show()

## NJT

In [19]:
import warnings
warnings.filterwarnings('ignore')
import anjl
from scipy.spatial.distance import squareform
import allel

# df_kdr_origins = pd.read_csv(f"{wkdir}/results/kdr-origins/kdr_origins.csv", sep="\t", index_col=0)
# df_kdr_origins = df_kdr_origins.reset_index().rename(columns={'index':'sample_id'})

# find seg sites and remove highly missing sites
ac = geno.count_alleles()
seg = ac.is_segregating()
gn_seg = geno.compress(seg, axis=0)
missing_mask = gn_seg.is_missing().sum(axis=1) > gn_seg.shape[1] * 0.05
gn_seg = gn_seg.compress(~missing_mask, axis=0)

ac = allel.GenotypeArray(gn_seg).to_allele_counts(max_allele=3)
X = np.ascontiguousarray(np.swapaxes(ac.values, 0, 1))

dists = amp.multiallelic_diplotype_pdist(X, metric=amp.multiallelic_diplotype_mean_cityblock)
dists = squareform(dists)
df_samples = metadata.set_index('sample_id')
df = df_samples[['taxon', 'location']]

df_dist_matrix = pd.DataFrame(dists, index=df_samples.index.to_list(), columns=df_samples.index.to_list())
# pivot long 
df_dists = df_dist_matrix.stack().reset_index().set_axis('sample_id_x sample_id_y distance'.split(), axis=1)
# merge with metadata
df_dists = df_dists.merge(df, left_on='sample_id_x', right_index=True).merge(df, left_on='sample_id_y', right_index=True, suffixes=('_x', '_y'))
# remove self comparisons
df_dists = df_dists[df_dists['sample_id_x'] != df_dists['sample_id_y']]
# dedup
df_dists = df_dists.assign(dedup=np.array([''.join(sorted([a,b])) for a,b in zip(df_dists.sample_id_x, df_dists.sample_id_y)]).astype(str))
df_dists = df_dists.sort_values('sample_id_x').drop_duplicates('dedup').drop('dedup', axis=1)
# normalise distances
df_dists = df_dists.assign(location=lambda x: x.location_x + " | " + x.location_y).drop(['location_x', 'location_y'], axis=1)
df_grp_dists = df_dists.groupby('location').agg({'distance': 'mean'}).sort_values('distance').rename(columns={'distance': 'mean_distance'}).reset_index()
df_dists = df_dists.merge(df_grp_dists, on='location').assign(normalised_dist=lambda x: x.distance - x.mean_distance).sort_values('normalised_dist')

# get the 500 most distant samples and exclude highly irregular ones 
far_samples = df_dists.sort_values('normalised_dist', ascending=False)[:int(df_dists.shape[0] * 0.005)][['sample_id_x', 'sample_id_y']].values.flatten()
far_samples, far_counts = np.unique(far_samples, return_counts=True)
exclude_outliers = far_samples[far_counts > int(df_samples.shape[0] * 0.1)]
print(f"excluding extreme outliers from NJT", exclude_outliers)

dists = df_dist_matrix.drop(exclude_outliers, axis=0).drop(exclude_outliers, axis=1).values
leaf_data = df_samples.query("sample_id not in @exclude_outliers").reset_index()
#leaf_data = leaf_data.merge(df_kdr_origins[['sample_id', 'kdr_origin']], on='sample_id', how='left')

Z = anjl.dynamic_nj(dists)

for col in cohort_cols:
    fig = anjl.plot(
        Z,
        leaf_data=leaf_data,
        color=col,
        hover_name="sample_id",
        hover_data=cohort_cols,# + ['kdr_origin'],  
        color_discrete_map=color_mapping[col],
        marker_size=8
    )
    fig.write_image(f"{wkdir}/results/njt_{col}.png", scale=2)
    fig.show()

excluding extreme outliers from NJT ['Siaya_Delta_Alive_111']
